In [6]:
import yfinance as yf

token = 'AAPL'
start_date = '2015-01-01'

data = yf.download(token,start_date, interval='1d')
data.to_csv(f'{token}.csv')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500,24.347176,212818400
2015-01-05,27.072500,27.162500,26.352501,26.562500,23.661270,257142000
2015-01-06,26.635000,26.857500,26.157499,26.565001,23.663498,263188400
2015-01-07,26.799999,27.049999,26.674999,26.937500,23.995314,160423600
2015-01-08,27.307501,28.037500,27.174999,27.972500,24.917267,237458000
...,...,...,...,...,...,...
2024-11-22,228.059998,230.720001,228.059998,229.869995,229.869995,38168300
2024-11-25,231.460007,233.250000,229.740005,232.869995,232.869995,90152800
2024-11-26,233.330002,235.570007,233.330002,235.059998,235.059998,45986200


In [2]:
import pandas as pd 

data = pd.read_csv('BTC-USD.csv', parse_dates=['Date'], index_col=['Date'])
data.drop(['Adj Close', 'Open', 'High', 'Low', 'Volume'], axis=1, inplace=True)

In [3]:
import ta

data['sma'] = ta.trend.sma_indicator(data['Close'], window=30)
data['ema20'] = ta.trend.ema_indicator(data['Close'], window=20)
data['ema50'] = ta.trend.ema_indicator(data['Close'], window=50)
data['ema200'] = ta.trend.ema_indicator(data['Close'], window=200)


# Añadir RSI
data['rsi'] = ta.momentum.rsi(data['Close'], window=14)

# Añadir MACD
macd = ta.trend.MACD(data['Close'], window_slow=26, window_fast=12, window_sign=9)
data['macd'] = macd.macd()
data['macd_signal'] = macd.macd_signal()

#Precio Futuro
data['Close_future'] = data['Close'].shift(-1)
# Dirección del Movimiento de Precio
data['Price_Up'] = (data['Close_future'] > data['Close']).astype(int)


#Boramos datos nulos
data.dropna(inplace=True)

data.to_csv('BTC-Indicators.csv',index=['Date'])
data = pd.read_csv('BTC-Indicators.csv', parse_dates=['Date'], index_col=['Date'])

In [ ]:
features = ['Close', 'sma', 'ema20', 'ema50', 'ema200', 'rsi', 'macd', 'macd_signal']
target = ['Price_Up']
# Separar X e y
X = data[features]
y = data[target]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle= False)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
